In [2]:
%%capture
!pip install feature-engine
!pip install optuna

In [3]:
%%capture
!pip install fastai wwf -q --upgrade

In [4]:

import torch
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader,Dataset
from torch.autograd import Variable
import torch.optim as optim
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances
from optuna.integration import FastAIV2PruningCallback

import warnings
warnings.filterwarnings("ignore")


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sb
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import random
from feature_engine.encoding import OneHotEncoder as fe_OneHotEncoder

import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
import gc

from fastai.tabular.all import *
#hide_input
from wwf.utils import state_versions
state_versions(['fastai', 'fastcore', 'wwf'])



---
This article is also a Jupyter Notebook available to be run from the top down. There
will be code snippets that you can then run in any environment.

Below are the versions of `fastai`, `fastcore`, and `wwf` currently running at the time of writing this:
* `fastai` : 2.7.11 
* `fastcore` : 1.5.28 
* `wwf` : 0.0.16 
---

In [5]:
train_raw = pd.read_csv('../wids_datathon2023/train_data.csv', parse_dates=["startdate"])
test_raw = pd.read_csv('../wids_datathon2023/test_data.csv', parse_dates=["startdate"])
submit = pd.read_csv('../wids_datathon2023/sample_solution.csv')
target = 'contest-tmp2m-14d__tmp2m'

In [6]:
def location_nom(train, test):
    scale = 14

    train.loc[:,'lat']=round(train.lat,scale)
    train.loc[:,'lon']=round(train.lon,scale)
    test.loc[:,'lat']=round(test.lat,scale)
    test.loc[:,'lon']=round(test.lon,scale)

    all_df = pd.concat([train, test], axis=0)
    all_df['loc_group'] = all_df.groupby(['lat','lon']).ngroup()
    train = all_df.iloc[:len(train)]
    test = all_df.iloc[len(train):].drop(target, axis=1)
    
    return train, test

def categorical_encode(train, test):
    le = LabelEncoder()
    train['climateregions__climateregion'] = le.fit_transform(train['climateregions__climateregion'])
    test['climateregions__climateregion'] = le.transform(test['climateregions__climateregion'])
    return train, test

def one_hot_categorical_encode(train, test):
    ohe_enc = fe_OneHotEncoder(
    top_categories=None,
    variables=['climateregions__climateregion'],  # we can select which variables to encode
    drop_last=True)  # to return k-1, false to return k
    
    ohe_enc.fit(train)
    train = ohe_enc.transform(train)
    test = ohe_enc.transform(test)
    return train, test
    
def creat_new_featute(df):
    df['year'] = df['startdate'].dt.year
    df['month'] = df['startdate'].dt.month
    df['day_of_year'] = df['startdate'].dt.dayofyear
    return df

def filter_na_cols(df):
    count_na_df = df.isna().sum() 
    if count_na_df[count_na_df > 0].tolist():
        return count_na_df[count_na_df > 0]
    else:
        return 'Clean dataset'

def feature_engineering(train_raw, test_raw):
    train, test = location_nom(train_raw, test_raw)
    #train = fill_na(train)
    train = creat_new_featute(train)
    test = creat_new_featute(test)
    #train, test = categorical_encode(train, test)
    drop_cols = ['index', 'startdate', 'lat', 'lon', target]
    features = [col for col in train.columns if col not in drop_cols]
    X = train[features]
    X_test = test[features]
    y = train[target]
    
    X, X_test = one_hot_categorical_encode(X, X_test)

    return X, y, X_test

In [7]:
X, y, X_test = feature_engineering(train_raw.copy(), test_raw.copy())

In [8]:
col_na = [
    'nmme0-tmp2m-34w__ccsm30', 
    'nmme-tmp2m-56w__ccsm3', 
    'nmme-prate-34w__ccsm3', 
    'nmme0-prate-56w__ccsm30', 
    'nmme0-prate-34w__ccsm30', 
    'nmme-prate-56w__ccsm3', 
    'nmme-tmp2m-34w__ccsm3']

g_means =  ['nmme0-tmp2m-34w__nmme0mean', 
 'nmme-tmp2m-56w__nmmemean', 
 'nmme-prate-34w__nmmemean', 
 'nmme0-prate-56w__nmme0mean', 
 'nmme0-prate-34w__nmme0mean', 
 'nmme-prate-56w__nmmemean', 
 'nmme-tmp2m-34w__nmmemean']


g_1 = ['nmme0-tmp2m-34w__cancm30',
'nmme0-tmp2m-34w__cancm40',
'nmme0-tmp2m-34w__ccsm40',
'nmme0-tmp2m-34w__cfsv20',
'nmme0-tmp2m-34w__gfdlflora0',
'nmme0-tmp2m-34w__gfdlflorb0',
'nmme0-tmp2m-34w__gfdl0',
'nmme0-tmp2m-34w__nasa0']

g_2 = ['nmme-tmp2m-56w__cancm3',
'nmme-tmp2m-56w__cancm4',
'nmme-tmp2m-56w__ccsm4',
'nmme-tmp2m-56w__cfsv2',
'nmme-tmp2m-56w__gfdl',
'nmme-tmp2m-56w__gfdlflora',
'nmme-tmp2m-56w__gfdlflorb',
'nmme-tmp2m-56w__nasa']

g_3 = ['nmme-prate-34w__cancm3',
'nmme-prate-34w__cancm4',
'nmme-prate-34w__ccsm4',
'nmme-prate-34w__cfsv2',
'nmme-prate-34w__gfdl',
'nmme-prate-34w__gfdlflora',
'nmme-prate-34w__gfdlflorb',
'nmme-prate-34w__nasa']

g_4 = [ 'nmme0-prate-56w__cancm30',
'nmme0-prate-56w__cancm40',
'nmme0-prate-56w__ccsm40',
'nmme0-prate-56w__cfsv20',
'nmme0-prate-56w__gfdlflora0',
'nmme0-prate-56w__gfdlflorb0',
'nmme0-prate-56w__gfdl0',
'nmme0-prate-56w__nasa0']

g_5 = ['nmme0-prate-34w__cancm30',
'nmme0-prate-34w__cancm40',
'nmme0-prate-34w__ccsm40',
'nmme0-prate-34w__cfsv20',
'nmme0-prate-34w__gfdlflora0',
'nmme0-prate-34w__gfdlflorb0',
'nmme0-prate-34w__gfdl0',
'nmme0-prate-34w__nasa0']

g_6 = ['nmme-prate-56w__cancm3',
'nmme-prate-56w__cancm4',
'nmme-prate-56w__ccsm4',
'nmme-prate-56w__cfsv2',
'nmme-prate-56w__gfdl',
'nmme-prate-56w__gfdlflora',
'nmme-prate-56w__gfdlflorb',
'nmme-prate-56w__nasa']

g_7 = ['nmme-tmp2m-34w__cancm3',
'nmme-tmp2m-34w__cancm4',
'nmme-tmp2m-34w__ccsm4',
'nmme-tmp2m-34w__cfsv2',
'nmme-tmp2m-34w__gfdl',
'nmme-tmp2m-34w__gfdlflora',
'nmme-tmp2m-34w__gfdlflorb',
'nmme-tmp2m-34w__nasa']

In [9]:
gs = [g_1, g_2, g_3, g_4, g_5, g_6, g_7]

zip_cols = zip(col_na, gs, g_means)
for c, g, m in (zip_cols):
    X[c] = (X[m]*9) - X[g].sum(1)
X['ccsm30'] = 9*X['nmme0mean'] - (X['cancm30']+X['cancm40']+X['ccsm40']+X['cfsv20']+X['gfdlflora0']+X['gfdlflorb0']+X['gfdl0']+X['nasa0'])

filter_na_cols(X)

'Clean dataset'

In [10]:
cols_clim =[]
for col in X.columns:
    if 'climateregions__climateregion' in col:
        cols_clim.append(col)

In [11]:
cols_sea_temp = []
for col in X.columns:
    if 'sst' in col:
        cols_sea_temp.append(col)

cols_wind = []
for col in X.columns:
    if ('wind' in col) and ('contest' not in col):
        cols_wind.append(col)
        
cols_prate = []
for col in X.columns:
    if 'prate' in col:
        cols_prate.append(col)

cols_14d = []
for col in X.columns:
    if '14d' in col:
        cols_14d.append(col)

cols_clim = cols_clim +['elevation__elevation']
cols_time_s= ['month','day_of_year','year']

cols_red = cols_sea_temp+ cols_wind+ cols_prate+ cols_14d+cols_clim+cols_time_s

In [12]:
len(cols_red)

202

In [13]:

X_reduced = X.loc[:,cols_red]
X_test_reduced = X_test.loc[:,cols_red]

In [14]:
X_train, X_val, y_train, y_val = train_test_split(X_reduced, y, test_size=0.33, random_state=42)

In [15]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    #os.environ['CUDA_LAUNCH_BLOCKING'] = 1
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_everything(seed=777)

In [16]:
def rmse_fast(preds, targs):
    x = (targs-preds)
    return (x**2).mean().sqrt()

## Optuna optimization


In [17]:
X_train, X_val, y_train, y_val = train_test_split(X_reduced, y, test_size=0.33, random_state=42)

In [18]:
train_index = X_train.index
val_index = X_val.index

splits = (L(list(train_index)), L(list(val_index)))

In [19]:
train_df_fast = pd.concat([X_reduced,y],axis=1)
test_df_fast = X_test_reduced

In [20]:
categorical_features_indices = np.where((X_reduced.dtypes != np.float64))

list_cols= list(X_reduced.columns)
categ_feats = []
for i in categorical_features_indices[0]:
    categ_feats.append(list_cols[i])

cont_feats = list(X_reduced.columns).copy()
for col in categ_feats:
    cont_feats.remove(col)

In [21]:
train_df_fast[categ_feats] = train_df_fast[categ_feats].astype('category')
test_df_fast[categ_feats] = test_df_fast[categ_feats].astype('category')

In [22]:
dpath = Path('../fast_ai_regression/OptunaFastAi')
dpath.mkdir(exist_ok=True, parents=True)

In [23]:
learner = None

lpath = Path(dpath/"best_learner.pkl")

if lpath.exists():
    best_learner = joblib.load(lpath)
else:
    best_learner = None

In [58]:
def objective(trial: optuna.Trial):
    num_layers = trial.suggest_int('n_layers', 2, 7)
    num_layers += 1
    layers, ps = [], []
    
    pot_layers = [
        [500, 1000, 1500, 2000, 2500,  3000, 3500, 4000, 5000],
        [100, 250, 500, 750, 1000, 1500, 2000, 2500, 3000],
        [50, 100, 200, 300, 400, 500, 750, 1000, 1500, 2000],
        [50, 100, 150, 200, 300, 400, 500, 750, 1000],
        [50, 100, 200, 300, 400, 500],
        [50, 100, 200, 300, 400],
        [50, 100, 200, 300]
    ]

    for i in range(num_layers - 1):
        num_units = trial.suggest_categorical(f'num_units_{i}', pot_layers[i])
        
        p = trial.suggest_uniform(f'ps_{i}', 0, 0.5)
        
        layers.append(num_units)
        ps.append(p)
    

    splits = (L(list(train_index)), L(list(val_index)))

    dls = TabularPandas(
        train_df_fast,
        cont_names=cont_feats,
        cat_names=categ_feats,
        procs=[Categorify, Normalize],
        y_names=target,
        splits=splits
    ).dataloaders(bs=1024)


    callbacks = [
        SaveModelCallback(min_delta=0.0005, monitor='_rmse', comp=np.less, fname='model_triv_best'),
        FastAIV2PruningCallback(trial, monitor='_rmse')
    ]

    emb_drop = trial.suggest_uniform('emb_drop', 0, 0.35)
    
    cfg = tabular_config(embed_p=emb_drop, ps=ps)
    global learner
    learner = tabular_learner(dls, layers=layers, metrics=[rmse], config=cfg)
    
    
    lr_max = trial.suggest_uniform('lr_max', 0.01, 0.2)
    weight_decay = trial.suggest_uniform('weight_decay', 0.02, 0.25)
    learner.fit_one_cycle(55, lr_max=lr_max, wd=weight_decay, cbs=callbacks)

    return learner.validate()[-1]

- This callback stores the best model in the global variable best_learner, so I can use it later without retraining.

In [59]:
def saveBestModelCallback(study, trial):
    global best_learner
    if study.best_trial == trial:
        best_learner = learner

In [60]:
spath = dpath / "study.pkl"
if spath.exists():
    study = joblib.load(spath)
else:
    study = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=4, interval_steps=4))

[I 2023-02-26 08:59:28,552] A new study created in memory with name: no-name-6139ec5f-0db7-4977-879f-f318beac2cab


In [61]:
study.optimize(objective, timeout=60*60*2, callbacks=[saveBestModelCallback])

epoch,train_loss,valid_loss,_rmse,time
0,5.295856,1.594501,1.262735,00:03
1,3.128030,1.405226,1.185422,00:03
2,3.049221,2.072742,1.439702,00:03
3,3.051765,3.241106,1.800307,00:03
4,3.032530,2.069721,1.438653,00:03
5,3.201805,2.568251,1.602576,00:03
6,3.193732,2.280755,1.510217,00:03
7,3.598706,3.513463,1.874423,00:03
8,3.435707,2.067247,1.437792,00:03
9,3.556163,128937.687500,359.078949,00:03


Better model found at epoch 0 with _rmse value: 1.2627354860305786.
Better model found at epoch 1 with _rmse value: 1.1854219436645508.
Better model found at epoch 43 with _rmse value: 1.1191030740737915.
Better model found at epoch 45 with _rmse value: 1.0481765270233154.
Better model found at epoch 47 with _rmse value: 0.9786047339439392.
Better model found at epoch 48 with _rmse value: 0.9524433016777039.
Better model found at epoch 49 with _rmse value: 0.9064294099807739.
Better model found at epoch 50 with _rmse value: 0.8858413696289062.
Better model found at epoch 51 with _rmse value: 0.8040922284126282.
Better model found at epoch 52 with _rmse value: 0.7766328454017639.
Better model found at epoch 53 with _rmse value: 0.765622615814209.


[I 2023-02-26 09:03:19,355] Trial 0 finished with value: 0.765622615814209 and parameters: {'n_layers': 3, 'num_units_0': 3000, 'ps_0': 0.08972584675383516, 'num_units_1': 100, 'ps_1': 0.38177531220484906, 'num_units_2': 50, 'ps_2': 0.11271241024102086, 'emb_drop': 0.15107620842341268, 'lr_max': 0.1714638486330908, 'weight_decay': 0.09030958029867461}. Best is trial 0 with value: 0.765622615814209.


epoch,train_loss,valid_loss,_rmse,time
0,7.503362,1.992918,1.411708,00:04
1,3.048896,2.351257,1.533381,00:04
2,2.389580,1.717666,1.310598,00:03
3,2.300182,2.763126,1.662265,00:04
4,2.266822,3.040839,1.743800,00:03
5,2.441931,1.745325,1.321108,00:03
6,2.448815,2.873997,1.695287,00:03
7,2.571877,1.950525,1.396612,00:03
8,2.583699,1.784189,1.335736,00:03
9,2.545008,1.871879,1.368166,00:03


Better model found at epoch 0 with _rmse value: 1.4117077589035034.
Better model found at epoch 2 with _rmse value: 1.3105977773666382.
Better model found at epoch 14 with _rmse value: 1.2505602836608887.
Better model found at epoch 26 with _rmse value: 1.2245396375656128.
Better model found at epoch 33 with _rmse value: 1.214733362197876.
Better model found at epoch 36 with _rmse value: 1.0725270509719849.
Better model found at epoch 42 with _rmse value: 1.0217788219451904.
Better model found at epoch 43 with _rmse value: 0.9858769178390503.
Better model found at epoch 45 with _rmse value: 0.9202377796173096.
Better model found at epoch 46 with _rmse value: 0.861053466796875.
Better model found at epoch 50 with _rmse value: 0.7482978105545044.


...

Better model found at epoch 0 with _rmse value: 1.3253819942474365.
Better model found at epoch 3 with _rmse value: 1.1217504739761353.
Better model found at epoch 6 with _rmse value: 1.0053205490112305.
Better model found at epoch 11 with _rmse value: 0.9046412706375122.
Better model found at epoch 13 with _rmse value: 0.8524971008300781.
Better model found at epoch 15 with _rmse value: 0.7695603370666504.
Better model found at epoch 24 with _rmse value: 0.7284788489341736.
Better model found at epoch 32 with _rmse value: 0.6835520267486572.
Better model found at epoch 34 with _rmse value: 0.6175066232681274.
Better model found at epoch 36 with _rmse value: 0.5310081243515015.
Better model found at epoch 39 with _rmse value: 0.4818590581417084.
Better model found at epoch 44 with _rmse value: 0.38855597376823425.
Better model found at epoch 45 with _rmse value: 0.3714657723903656.
Better model found at epoch 47 with _rmse value: 0.3528395891189575.
Better model found at epoch 48 with 

[I 2023-02-26 10:56:38,794] Trial 33 finished with value: 0.3031017482280731 and parameters: {'n_layers': 2, 'num_units_0': 1500, 'ps_0': 0.11416107210165254, 'num_units_1': 2000, 'ps_1': 0.23051586910594118, 'emb_drop': 0.29160795485358487, 'lr_max': 0.022000354388507115, 'weight_decay': 0.020141650688437627}. Best is trial 32 with value: 0.25463512539863586.


epoch,train_loss,valid_loss,_rmse,time
0,7.988427,1.985591,1.409110,00:05
1,3.313164,2.276115,1.508680,00:05
2,2.453650,1.590806,1.261272,00:05
3,2.110280,1.394956,1.181083,00:05
4,2.059320,1.185915,1.088997,00:05
5,1.836374,1.162728,1.078299,00:05
6,2.076607,2.618379,1.618141,00:05
7,1.801160,1.130070,1.063048,00:05
8,1.735487,1.256697,1.121025,00:05
9,2.026609,0.965343,0.982519,00:05


Better model found at epoch 0 with _rmse value: 1.4091100692749023.
Better model found at epoch 2 with _rmse value: 1.2612717151641846.
Better model found at epoch 3 with _rmse value: 1.1810826063156128.
Better model found at epoch 4 with _rmse value: 1.0889971256256104.
Better model found at epoch 5 with _rmse value: 1.0782989263534546.
Better model found at epoch 7 with _rmse value: 1.0630475282669067.
Better model found at epoch 9 with _rmse value: 0.98251873254776.
Better model found at epoch 29 with _rmse value: 0.9063948392868042.


[I 2023-02-26 10:59:52,753] Trial 34 pruned. Exception occured in `FastAIV2PruningCallback` when calling event `after_fit`:
	Trial was pruned at epoch 32.


In [62]:
study.best_params

{'n_layers': 3,
 'num_units_0': 2000,
 'ps_0': 0.07518389581642176,
 'num_units_1': 2000,
 'ps_1': 0.23794403763060212,
 'num_units_2': 1500,
 'ps_2': 0.44882134370830074,
 'emb_drop': 0.32719414708399197,
 'lr_max': 0.021051175419473198,
 'weight_decay': 0.020695103616881455}

## Predictions (test set)


In [63]:
test_dl = best_learner.dls.test_dl(test_df_fast)

preds_fast_tuned_raw = best_learner.get_preds(dl=test_dl)[0]
preds_fast_tuned = preds_fast_tuned_raw.squeeze().numpy()

In [64]:
preds_fast_tuned

array([29.108852 , 29.198254 , 29.21104  , ...,  5.2241263,  6.327905 ,
        7.2269735], dtype=float32)

In [66]:
# save the model
best_learner.export('../fast_ai_regression/fast_tuned_autresfeats')

In [26]:
test_df_fast.shape

(31354, 202)

In [24]:
# Loading the saved model 
# Requirements:  pandas == 1.4.3, Python == 3.8 or higher, ideally 3.9.13)
# You can find the model here (I also added it as a dataset input to my Kaggle notebook (Part 4)).

learn = load_learner('../fast_ai_regression/fast_tuned_autresfeats')
dl_test = learn.dls.test_dl(test_df_fast)

preds_fast_raw = learn.get_preds(dl=dl_test)[0]
preds_fast = preds_fast_raw.squeeze().numpy()
preds_fast

array([29.108852 , 29.198254 , 29.211044 , ...,  5.224127 ,  6.3279047,
        7.2269735], dtype=float32)

In [25]:
best_631_keras_tabnet = pd.read_csv('../Pytorch&Keras/sub_keras/(0.631)best637_0.9_0.1keras_1.csv')[target]

In [26]:
0.9*best_631_keras_tabnet + 0.1*preds_fast# gives 0.627 (LB)

0        29.256436
1        29.258618
2        29.277761
3        29.335652
4        29.397711
           ...    
31349     6.621614
31350     6.811817
31351     6.192805
31352     7.006481
31353     7.527265
Name: contest-tmp2m-14d__tmp2m, Length: 31354, dtype: float64